In [1]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=9882bd3ed7fe393a9ae147a0169710f24685909be7d20f86f3b792f185fbbcf8
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


# Imports

In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.datasets as datasets
from efficientnet_pytorch import EfficientNet

# Define data transforms (Augmentation)

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Download CIFAR-10 dataset

In [23]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
valset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_val)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split trainset into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Train Val Data Split

In [5]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# Define indices for stratified sampling
indices = np.arange(len(trainset))
targets = np.array(trainset.targets)

# Stratified split for train-validation sets
stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, val_indices = next(stratified_splitter.split(indices, targets))

# Create train and validation datasets using the sampled indices
train_dataset = torch.utils.data.Subset(trainset, train_indices)
val_dataset = torch.utils.data.Subset(trainset, val_indices)

print(f"Number of samples in train set: {len(train_dataset)}")
print(f"Number of samples in validation set: {len(val_dataset)}")


Number of samples in train set: 40000
Number of samples in validation set: 10000


# Define Data Loaders

In [6]:
k = 256

trainloader = DataLoader(train_dataset, batch_size=k, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=k, shuffle=False)
testloader = DataLoader(testset, batch_size=k, shuffle=False)

# Load Pretrained Models

In [16]:
# Load pretrained ResNet-50
resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)  # Change the output layer to match CIFAR-10 classes

# Load pretrained EfficientNet-b0
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
num_ftrs = efficientnet._fc.in_features
efficientnet._fc = nn.Linear(num_ftrs, 10)  # Change the output layer to match CIFAR-10 classes

Loaded pretrained weights for efficientnet-b0


# Define Loss Function, Optimizer and learning rate scheduler

In [17]:
criterion = nn.NLLLoss()
lr = 0.001
optimizer_resnet = optim.SGD(resnet.parameters(), lr=lr, momentum=0.9)
optimizer_efficientnet = optim.SGD(efficientnet.parameters(), lr=lr, momentum=0.9)

scheduler_resnet = optim.lr_scheduler.StepLR(optimizer_resnet, step_size=3, gamma=0.1)
scheduler_efficientnet = optim.lr_scheduler.StepLR(optimizer_efficientnet, step_size=3, gamma=0.1)

# Define Training function

In [18]:
def train_model(model, criterion, optimizer, scheduler, trainloader, valloader, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valloader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        # Adjust learning rate
        scheduler.step()

# Training our models

In [ ]:
# Train ResNet-50
print("Training ResNet-50...")
train_model(resnet, criterion, optimizer_resnet, scheduler_resnet, trainloader, valloader)

# Train EfficientNet-b0
print("\nTraining EfficientNet-b0...")
train_model(efficientnet, criterion, optimizer_efficientnet, scheduler_efficientnet, trainloader, valloader)

# Define function to calculate test accuracy

In [20]:
def test_accuracy(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Testing our models

In [ ]:
# Test ResNet-50
resnet_test_accuracy = test_accuracy(resnet, testloader)
print(f"Test Accuracy of ResNet-50: {resnet_test_accuracy:.2f}%")

# Test EfficientNet-b0
efficientnet_test_accuracy = test_accuracy(efficientnet, testloader)
print(f"Test Accuracy of EfficientNet-b0: {efficientnet_test_accuracy:.2f}%")

# Saving our models 

In [ ]:
# After training your model...
torch.save(resnet.state_dict(), 'resnet50_model.pth')
torch.save(efficientnet.state_dict(), 'efficientnet_b0_model.pth')

# Using our saved model of ResNet-50

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
from PIL import Image

image_path = '/content/11.png'
image = Image.open(image_path).convert('RGB')  # Ensure RGB format

In [ ]:
import torch
import torch.nn as nn
import torchvision

def get_resnet50():
    model = torchvision.models.resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)  # 10 classes for CIFAR10
    return model

In [ ]:
model = get_resnet50()  # Use the same initialization function as during training
model.load_state_dict(torch.load('resnet50_model.pth')) 
model.eval()  # Set the model to evaluation mode

In [ ]:
image_tensor = transform(image).unsqueeze(0)  # Add a batch dimension

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_tensor = image_tensor.to(device)  

with torch.no_grad():
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs.data, 1) 

In [ ]:
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]  # Your list of CIFAR10 classes
print(f"Predicted Class: {classes[predicted.item()]}") 

# Using our saved model of EfficientNet-b0

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image

def get_efficientnet():
    model = torchvision.models.efficientnet_b0(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, 10) # 10 classes for CIFAR10
    return model

In [ ]:
# Preprocessing transforms (match your training setup)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
model = get_efficientnet()
model.load_state_dict(torch.load('efficientnet_b0_model.pth'))
model.eval()  # Set to evaluation mode

In [ ]:
image_path = '/content/11.png'
image = Image.open(image_path).convert('RGB')  
image_tensor = transform(image).unsqueeze(0)  

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_tensor = image_tensor.to(device) 

with torch.no_grad():
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs.data, 1) 

In [ ]:
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
print(f"Predicted Class: {classes[predicted.item()]}") 